In [ ]:
import os
import tweepy as tw
import pandas as pd
import re
import numpy as np
import json

In [ ]:
def get_keys(path):
    with open(path) as f:
        return json.load(f)

In [ ]:
keys = get_keys("/Users/Andy/.secret/twitter_api.json")

In [ ]:
consumer_key = keys['consumer_key']
consumer_secret = keys['consumer_secret']
access_token = keys['access_token']
access_token_secret = keys['access_token_secret']


In [ ]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
twitter_accounts = ["from:XboxSupport", "from:UPSHelp", "from:JetBlue", "from:NikeSupport", "from:ComcastCares",
                   "from:AmazonHelp", "from:Zappos_Service", "from:AskTarget", "from:Tesco", "from:Lululemon"]

In [ ]:
#Define the search term and the date_since date as variables
search_words = "from:XboxSupport"
date_since = "2020-1-13"

In [ ]:
#new_search = search_words + "-filter:links -filter:retweets -filter:media -filter:native_video -filter:images -filter:twimg -filter:replies"
new_search = search_words

new_search

In [ ]:
tweet_list = []
for account in twitter_accounts:
    tweets = tw.Cursor(api.search,
                      q=account,
                      lang='en',).items(10)
    new_tweets = [tweet.text for tweet in tweets]
    tweet_list += new_tweets
    print(account)

In [ ]:
tweet_list

In [ ]:
len(tweet_list)

In [ ]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)        
    return input_txt
def clean_tweets(lst):
    # remove twitter Return handles (RT @xxx:)
    lst = np.vectorize(remove_pattern)(lst, "RT @[\w]*:")
    # remove twitter handles (@xxx)
    lst = np.vectorize(remove_pattern)(lst, "@[\w]*")
    # remove URL links (httpxxx)
    lst = np.vectorize(remove_pattern)(lst, "https?://[A-Za-z0-9./]*")
    # remove special characters, numbers, punctuations (except for #)
    lst = np.core.defchararray.replace(lst, "[^a-zA-Z#]", " ")
    return lst

tweets = clean_tweets(tweet_list)

In [ ]:
tweets

In [ ]:
import csv

In [ ]:
with open('data_wendys.csv', 'w', newline='' , encoding='utf-8') as myfile:
    wr = csv.writer(myfile, delimiter=',')
    wr.writerow(tweets)